# 🦜🔗 解读LangChain核心源码：关于LLM和Agent（上）

# 课程开始

## 这节课会带给你

- 🌹 一起阅读 Langchain 相关组件的源码，以便更好地阅读官方文档
- 🌹 掌握 Langchain 文档中未曾提及、翻看源码才知晓的实用技巧
- ✍️ 从零开始集成智谱大模型到 LangChain：解决智谱官方SDK不兼容的问题
- ✍️ 按 LangChain 框架自定义一个智能体：再现《手撕AutoGPT》中的智能体
- ✍️ 最终实践：再现《手撕AutoGPT》：langchain+自定义大模型+自定义智能体

## 涉及的资源

### ❤️ LangChain 官网

[langchain官网的python文档](https://python.langchain.com/docs/)

### ❤️ LangChain 模块源码概览

[🔗 查看核心源码资源](https://github.com/langchain-ai/langchain/tree/master/libs)

| 源码位置 | 功能描述 |
| :--- | :--- |
| [langchain/libs/langchain/langchain](https://github.com/langchain-ai/langchain/tree/master/libs/langchain/langchain) | 模块入口，会导入core、community等其他模块 |
| [langchain/libs/core/langchain_core](https://github.com/langchain-ai/langchain/tree/master/libs/core/langchain_core) | 核心组件和关键的基类实现 |
| [langchain/libs/partners/openai/langchain_openai](https://github.com/langchain-ai/langchain/tree/master/libs/partners/openai/langchain_openai) | 合作伙伴（官方合作）组件 |
| [langchain/libs/community/langchain_community](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community) | 社区（非官方）组件 |
| [langchain/libs/experimental/langchain_experimental](https://github.com/langchain-ai/langchain/tree/master/libs/experimental/langchain_experimental) | 试验性功能（前沿探索组件，不对版本稳定做承诺） |


### 🌹 推荐阅读：从 Langchain 的最核心组件 Runnable 开始阅读源码

[🔗 查看 langchain_core/runnables/base.py#L104](https://github.com/langchain-ai/langchain/blob/ad77fa15eec4dae431171b7e8c13b8c1f9edec98/libs/core/langchain_core/runnables/base.py#L104)

**（1）Runnable实用方法：**

- Runnable
    - assign()
    - bind()
    - with_config()
    - get_name()
    - get_graph()
    - get_prompts()
    - input_schema
    - output_schema
    - invoke / ainvoke / batch / abatch 
    - stream /astream / astream_log / astream_events

----
**（2）RunnableSerializable实用方法：**

- Runnable
    - RunnableSerializable
        - dumps()
        - loads()
        - first()
        - last()
        - middle()

----
**（3）配置能力子类：**

- Runnable
    - RunnableSerializable
        - RunnableBindingBase
            - RunnableBinding（向Runnable实例传递参数）
        - DynamicRunnable
            - RunnableConfigurableFields
            - RunnableConfigurableAlternatives

----
**（4）流程控制子类：**

- Runnable
    - RunnableSerializable
        - RunnablePassthrough（传递额外输入）
        - RunnableSequence（实现顺序执行，可以用重载的`|`符号或`RunnableSequence`来构造）
        - RunnableParallel（实现并行执行，可以用`Dict`或`RunnableParallel`类来构造，别名`RunnableMap`）

----
**（5）大模型子类：**

- Runnable
    - RunnableSerializable
        - BaseLanguageModel
            - BaseLLM（派生其他大模型）
                - BaseOpenAI（派生其他大模型）
                    - OpenAI
                - LLM
                    - ...
                - ...
            - BaseChatModel
                - ChatOpenAI
                - ...

----
**（6）提示语子类：**
- Runnable
    - RunnableSerializable
        - BasePromptTemplate `[Dict, PromptValue]`
            - StringPromptTemplate（字符串模板）
            - BaseChatPromptTemplate（对话模板）
            - ImagePromptTemplate
            - PipelinePromptTemplate

----
**（7）检索器子类：**
- Runnable
    - RunnableSerializable
        - BaseRetriever `[RetrieverInput, RetrieverOutput]`（派生各类检索器）

----
**（8）Tool子类：**
- Runnable
    - RunnableSerializable
        - BaseTool `[Union[str, Dict], Any]`（派生各类工具）

----
**（9）输出解析子类：**
- Runnable
    - RunnableSerializable
        - BaseGenerationOutputParser `[Union[str, BaseMessage], T]`
        - BaseOutputParser（派生各类输出解析）

----
**（10）输入赋值子类：**
- Runnable
    - RunnableSerializable
        - RunnablePassthrough
        - RunnableAssign
        - RunnablePick

----
**（11）遗留Chain子类：**
- Runnable
    - RunnableSerializable
        - Chain（结构化Runnable）
            - AgentExecutor（执行智能体）

----
**（12）快速自定义Runnable的工具子类：**
- Runnable（）
    - RunnableGenerator（常用于处理输出可能是迭代器结果的chain）
    - RunnableLambda（常用于包装普通函数，装饰函数@chain）


### 🌹 本节（上半部份）涉及的 LangChain 源码

- 🌹 查看 Runnable：[langchain_core/runnables/base.py#L104](https://github.com/langchain-ai/langchain/blob/ad77fa15eec4dae431171b7e8c13b8c1f9edec98/libs/core/langchain_core/runnables/base.py#L104)
- 🌹 查看 BaseLanguageModel：[langchain_core/language_models/base.py](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/language_models/base.py#L74-L81)
- 🌹 查看 BaseChatModel：[langchain_core/language_models/chat_models.py](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/language_models/chat_models.py#L100)
- 🌹 查看 OpenAI 风格的消息转换： [langchain_community.adapters.openai](https://github.com/langchain-ai/langchain/blob/3f7da03dd89e76f290ce361c72d1676319110163/libs/community/langchain_community/adapters/openai.py#L112-L156)
- 🌹 查看对话消息语法糖：[langchain_core/messages/utils.py](https://github.com/langchain-ai/langchain/blob/c93d4ea91cfcf55dfe871931d42aa22562f8dae2/libs/core/langchain_core/messages/utils.py#L130-L168)
- 🌹 查看核心源码资源：[github.com/langchain-ai](https://github.com/langchain-ai/)


# （一）解读源码，集成自己的大模型到 langchain

<div class="alert alert-info">
    <b>干货从这里开始！</b><br>
    接下来的例子中，会穿插 langchian 源码解读。
</div>

In [ ]:
# 加载 .env 到环境变量
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

## 1、大模型的一般用法回顾

### ✍️ 使用 OpenAI

In [ ]:
# LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [ ]:
# invoke
text = "你知道「夏洛特烦恼」这部电影吗？"
response = llm.invoke(text)
print(response.content)

In [ ]:
# stream
for chunk in llm.stream(text):
    print(chunk.content, end="|", flush=True)

<div class="alert alert-warning">
    <b>⚠️ 思考</b><br>
    langchain 支持的8个方法都在什么场景下使用？
</div>


- invoke：最简单
- batch: 后台批量
- stream：让用户体验流式输出
- ainvoke / astream / abatch: 实现异步体验
- astream_log / astream_events: 从Runable、LCEL、智能体、langgraph等提取特定的流式输出

### ✍️ LCEL：LLM + Prompt + OutputParser

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser

prompt = PromptTemplate.from_template("你知道{name}这部电影吗?")
chain = prompt | llm | StrOutputParser()

for chunk in chain.stream({"name": "夏洛特烦恼"}):
    print(chunk, end="|", flush=True)

## 2、简单的开始：实现「楼下邻居老大爷」AI大模型

<div class="alert alert-info">
    <b>⚠️ 要深度使用 langchain 就必须阅读源码：</b><br>
    像集成自己的大模型到 langchain 这样的任务，langchain 文档中基本未曾提及。<br>
    因此，必须翻看源码才能搞清楚实现的机理。<br><br>
    但 langchain 的🌹魅力🌹就在于：你可以参与community，或建立自己的community ！
</div>

### 🦜 需求分析：参考电影片段，把「非常有智慧的楼下邻居老大爷」变成 AI大模型

这个例子，我们先讲清楚langchain的大模型标准实现框架，下个例子再讲如何对接实际的大模型。

1. 使用大模型：模拟一个大模型
2. 生成能力：提及马冬梅时生成打岔闲聊（马什么梅？马冬什么？什么冬梅？），其余生成“哦...“

![马什么梅？马冬什么？什么冬梅？](./madongmei.gif)

### ✍️ 基本实现：invoke

In [ ]:
import time
import re
from typing import Any, Dict, Iterator, List, Optional, Union
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import HumanMessage, HumanMessageChunk, AIMessage, AIMessageChunk, BaseMessage
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult

In [ ]:
class ChatWithOlderAI(BaseChatModel):
    """模拟跟马冬梅楼下邻居老大爷的对话"""

    # 必须实现
    @property
    def _llm_type(self) -> str:
        return "chat-with-neighber-older"

    # 必须实现
    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:
        generations = [ChatGeneration(message=res) for res in self._ask_remote(messages)]
        return ChatResult(generations=generations)

    # 有老大爷的智慧计时周期
    i: int = 0

    # 访问远程大模型
    # 当你需要实现自己的大模型时，主要是替换这部分
    def _ask_remote(self, messages: List[BaseMessage]) -> List[BaseMessage]:
        answers = [HumanMessage(m) for m in [
            "马什么梅？",
            "什么冬梅？？",
            "马东什么？？？",
        ]]
        
        if(re.search("马冬梅", messages[0].content)):
            response = answers[self.i]
            self.i = (self.i + 1) if self.i < (len(answers) - 1) else 0
        else:
            response = AIMessage("哦...")
            
        return [response]

In [ ]:
questions = [[HumanMessage(m)] for m in [
    "大爷，楼上322住的是马冬梅家吗？",
    "马冬梅啊",
    "马冬梅！",
    "我是说马冬梅！",
    "您歇着吧...",
]]

llm = ChatWithOlderAI()

for question in questions:
    print(f"\n\n夏洛：{question[0].content}")
    print("大爷：", end="")
    print(llm.invoke(question).content, end="")
    # for chunk in llm.stream(question):
    #     print(chunk.content, end="|")

### ✍️ 支持流式输出：stream

In [ ]:
class StreamChatWithOlderAI(ChatWithOlderAI):
    """模拟跟大爷的对话，支持流"""

    def _stream(
        self,
        messages: List[BaseMessage],
        *args,
        **kwargs: Any,
    ) -> Iterator[ChatGenerationChunk]:
        response = self._ask_remote(messages)
        for chunk in response[0].content:
            time.sleep(0.1)
            yield ChatGenerationChunk(message=AIMessageChunk(content=chunk))

In [ ]:
llm_stream = StreamChatWithOlderAI()

for question in questions:
    print(f"\n\n夏洛：{question[0].content}")
    print("大爷：", end="")
    for chunk in llm_stream.stream(question):
        print(chunk.content, end="|")

### ✍️  LCEL：Prompt + LLM + OutputParser

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser

questions_text = [
    "楼上322住的是马冬梅家吗？",
    "马冬梅啊",
    "马冬梅！",
    "我是说马冬梅！",
    "大爷您歇着吧..."
]

llm_stream = StreamChatWithOlderAI()
prompt = PromptTemplate.from_template("大爷，{question}")
chain = prompt | llm_stream | StrOutputParser()

for question in questions_text:
    print(f"\n\n夏洛：{question}")
    print("大爷：", end="")
    for chunk in chain.stream({"question": question}):
        print(chunk, end="|")

### 🌹 审视链上各节点的输入输出

In [ ]:
chain.get_graph().print_ascii()

In [ ]:
chain.first

In [ ]:
# llm_stream.input_schema.schema()
# chain.input_schema.schema()
# prompt.output_schema.schema()
# prompt.invoke({"question":"你好"})

### 🌹 阅读源码：实现 langchain 大模型的原理

#### （1）BaseLanguageModel

[🔗 查看 langchain_core/language_models/base.py](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/language_models/base.py#L74-L81)

```python
class BaseLanguageModel(
    RunnableSerializable[LanguageModelInput, LanguageModelOutputVar], ABC
):
    """Abstract base class for interfacing with language models.

    All language model wrappers inherit from BaseLanguageModel.
    """
```

#### （2）BaseChatModel

[🔗 查看 langchain_core/language_models/chat_models.py](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/language_models/chat_models.py#L100)

**核心逻辑：**

```python
class BaseChatModel(BaseLanguageModel[BaseMessage], ABC):
    """Base class for Chat models."""

    ...

    def invoke(...) -> BaseMessage:
        # ...
        self.generate_prompt(...)
        # generate_prompt >> generate >> _generate

    def ainvoke(...) -> BaseMessage:
        # ...
        self.agenerate_prompt(...)
        # agenerate_prompt >> agenerate >> _agenerate >> _generate
    
    def stream(...) -> Iterator[BaseMessageChunk]:
        # ...
        if type(self)._stream == BaseChatModel._stream:
            # model doesn't implement streaming, so use default implementation
            yield cast(
                BaseMessageChunk, self.invoke(input, config=config, stop=stop, **kwargs)
            )
        # ...

    async def astream(...) -> AsyncIterator[BaseMessageChunk]:
        # 在#19332合并中，_astream方法实现已经被简化
        # https://github.com/langchain-ai/langchain/pull/19332/commits/afbe6ac659e41ab5f4a6f4dcaa33511e9e59e4d5
        if (
            type(self)._astream is BaseChatModel._astream
            and type(self)._stream is BaseChatModel._stream
        ):
            # No async or sync stream is implemented, so fall back to ainvoke
            yield cast(
                BaseMessageChunk,
                await self.ainvoke(input, config=config, stop=stop, **kwargs),
            )
        # ...

    # bacth, abatch, astream_log, astream_events 
    # ...
```

**必须实现的部分：**

```python
    @property
    @abstractmethod
    def _llm_type(self) -> str:
        return "chat-with-neighber-older"
        
    ## ******** invoke / ainvoke / batch / abatch **********
    @abstractmethod
    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:
        """Top Level call"""

    ## ******** stream / astream / astream_log / astream_events **********
    def _stream(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> Iterator[ChatGenerationChunk]:
        raise NotImplementedError()        
```

### 🌹 推荐阅读：langchain 内置实现的 FakeLLM 源码

- [🔗 查看 FakeListLLM](https://github.com/langchain-ai/langchain/blob/8595c3ab59371d9932310eb12a2c3220afe3ba84/libs/core/langchain_core/language_models/fake.py#L14)
- [🔗 查看 FakeStreamingListLLM](https://github.com/langchain-ai/langchain/blob/8595c3ab59371d9932310eb12a2c3220afe3ba84/libs/core/langchain_core/language_models/fake.py#L61)
- [🔗 查看 FakeMessagesListChatModel](https://github.com/langchain-ai/langchain/blob/8595c3ab59371d9932310eb12a2c3220afe3ba84/libs/core/langchain_core/language_models/fake_chat_models.py#L16)
- [🔗 查看 FakeChatModel](https://github.com/langchain-ai/langchain/blob/8595c3ab59371d9932310eb12a2c3220afe3ba84/libs/core/langchain_core/language_models/fake_chat_models.py#L120C7-L120C20)

<div class="alert alert-success">
    <b>⚠️ 参考：</b><br>
    如果你觉得我们上面的例子还不够丰富，可以参考这几个Fake系列的大模型源码，可以参考上面的例子。
</div>

## 3、集成智谱大模型到 Langchain

### 🦜 需求分析：结合智谱官方文档，编写ChatZhipuAI

- [智谱AI官方的Python接口文档](https://maas.aminer.cn/dev/api#sdk)
- [Langchain中已有的智谱AI组件（旧版本）](https://python.langchain.com/docs/integrations/chat/zhipuai)

[官方API zhpuai v2.0.1 在2024年1月16日发布](https://pypi.org/project/zhipuai/#history)

<div class="alert alert-warning">
    <b>⚠️ 智谱AI遇到的尴尬：</b><br>
    智谱官方SDK升级到4.0之后, Langchain相应的包一直没有更新。<br>
    截止到2024年3月29日，仍然不可用（如果突然可用了也不要紧，本课内容原本就是抛转引玉）
</div>

#### （1）智谱官方可用接口
- 直接调用：可实现 invoke
- 异步调用（先调用，再查询结果）：适合实现 ainvoke / batch / abatch
- 流式调用（SSE，Server-Send Events)：适合实现 stream / astream / stream_log / stream_events

#### （2）智谱官方支持能力
- 支持工具：本地工具回调 / 云上检索 / 云上互联网搜索
- 支持识图
- 支持生图

<div class="alert alert-info">
    <b>⚠️ 思考：</b><br>
    能否实现一个自定义大模型，支持多模态能力？<br>
    input: List[Union[文字, 图像]] -> Union[文字, 图像]
</div>

#### （3）智谱官方速率限制

[🔗 智谱AI官方文档中对速率限制的说明](https://maas.aminer.cn/dev/howuse/rate-limits/why?tab=5)：

> 当前我们基于用户的月度 API 调用消耗金额情况将速率控制分为6种等级。
>
> 消耗金额选取逻辑：我们会选取用户当前月份1号～t-1日的调用 API 推理消耗总金额和用户上个月的 API 调用消耗总金额做比较，取更高金额作为用户当前的 API 消耗金额。
>
> 特别的，若您从未曾付费充值/购买过资源包，则会归为免费级别。

**整理GLM4模型使用限制如下：**
|用户等级|使用量|GLM4并发限制|
|:---|:---|:---|
|免费|api调用消耗0元-50元/每月（不含）|5|
|使用量1|api调用消耗50元-500元/每月（不含）|10|
|使用量2|api调用消耗500元-5000元/每月（不含）|20|
|使用量3|api调用消耗5000元-10000元/每月（不含）|30|
|使用量4|api调用消耗10000元-30000元/每月（不含）|100|
|使用量5|api调用消耗30000元以上/每月|200|

<div class="alert alert-warning">
    <b>⚠️ 思考：</b><br>
    能否实现一个自定义大模型，通过多个低用量账户池的管理机制来提高可用的调用速率？<br>
    速率限制属于服务端降级，调用时自动按照速率限制排队，影响用户体验。
</div>

### ✍️ 测试官方例子

看官方例子：[🔗 查看 https://github.com/MetaGLM/zhipuai-sdk-python-v4](https://github.com/MetaGLM/zhipuai-sdk-python-v4)

In [ ]:
from zhipuai import ZhipuAI

In [ ]:
## 看看官方的例子是否能正确运行
client = ZhipuAI()
response = client.chat.completions.create(
    model="glm-4",
    messages=[
        {"role": "user", "content": "你叫什么名字"},
    ],
)
print(response.choices[0].message)

### ✍️ 包装为一个函数调用（假装我不想直接用 langchain）

In [ ]:
def ask_zhipu(question: str) -> str:
    client = ZhipuAI()

    messages = [
        {"role": "system", "content": "你是一个翻译机器人，我说中文你就直接翻译成英文，我说英文你就直接翻译为中文。不要输出其他，不要啰嗦。"},
        {"role": "user", "content": "你好"},
        {"role": "assistant", "content": "hello"},
        {"role": "user", "content": question},
    ]

    response = client.chat.completions.create(
        model="glm-4",
        messages=messages,
    )
    return(response.choices[0].message.content)

ask_zhipu("你叫什么名字？")

### ✍️ 支持与 Prompt 协作

#### （1）能否实现如下场景？（似乎使用 langchain 也没那么坏）

```python
chain = prompt | llm | StrOutputParser()
chain.invoke({"question": "你叫什么名字？"})
```

基本思路：

Prompt输入（按langchain标准） <br>
⬇️ <br>
Prompt输出（按langchain标准） <br>
⬇️ <br>
LLM输入（按大模型标准） <br>
⬇️ <br>
LLM输出（按langchain标准）
⬇️ <br>
...

#### （2）构造 Prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个翻译机器人，我说中文你就直接翻译成英文，我说英文你就直接翻译为中文。不要输出其他，不要啰嗦。"),
    ("human", "你好"),
    ("ai", "hello"),
    ("human", "{question}"),
])
prompt.invoke({"question":"你叫什名字？"})

<div class="alert alert-warning">
    <b>⚠️ 思考</b><br>
    上面代码中， 为什么 from_messages 支持 system、human、ai 这些名字？还有其他名字吗？
</div>

**🌞 提示：**
- 文档中没有答案，需要研究源码：[🔗 查看 langchain_core/messages/utils.py](https://github.com/langchain-ai/langchain/blob/c93d4ea91cfcf55dfe871931d42aa22562f8dae2/libs/core/langchain_core/messages/utils.py#L130-L168)

#### （3）从 Prompt 输出格式，转换到大模型的输入格式

**探查格式：**

In [ ]:
prompt.invoke({"question":"你叫什名字？"}).to_messages()

**转换格式：**

这里可以借助 openai 标准的转换函数： [langchain_community.adapters.openai](https://github.com/langchain-ai/langchain/blob/3f7da03dd89e76f290ce361c72d1676319110163/libs/community/langchain_community/adapters/openai.py#L112-L156)，节省很多代码。

In [ ]:
from langchain_community.adapters.openai import convert_message_to_dict
from langchain_core.messages import HumanMessage, AIMessage

In [ ]:
[convert_message_to_dict(m) for m in prompt.invoke({"question":"你叫什名字？"}).to_messages()]

### ✍️ 直接用 RunnableLambda 达到目的（假装我不想用 langchain 的大模型基类）

In [ ]:
from langchain_core.runnables import chain
from typing import List
from langchain_core.prompt_values import ChatPromptValue 
from langchain.schema.output_parser import StrOutputParser

@chain
def ask_zhipu(promptValue: ChatPromptValue) -> AIMessage:
    client = ZhipuAI()
    response = client.chat.completions.create(
        model="glm-4",
        messages=[convert_message_to_dict(m) for m in promptValue.to_messages()],
    )
    return(AIMessage(response.choices[0].message.content))

In [ ]:
chain = prompt | ask_zhipu | StrOutputParser()
chain.invoke({"question": "你叫什名字？"})

In [ ]:
# 看看当前链的结构
chain.get_graph().print_ascii()

### ✍️ 基于 BaseChatModel 达到目的（假装我开始想获得 LCEL 的诸多好处）

#### （1）从大模型的输出格式，转换到 langchain 的标准输出格式

In [ ]:
from langchain_community.adapters.openai import convert_dict_to_message

#### （2）实现支持 invoke 的版本

In [ ]:
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.language_models.chat_models import BaseChatModel
from typing import Any, Dict, Iterator, List, Optional, cast, Mapping
from langchain_core.messages import BaseMessage
from langchain_core.outputs import ChatGeneration, ChatResult

In [ ]:
class MiniZhipuAI(BaseChatModel):
    """支持最新的智谱API"""

    client: Optional[ZhipuAI] = None

    def __init__(self, *args: Any, **kwargs: Any) -> None:
        super().__init__(*args, **kwargs)
        self.client = ZhipuAI()

    @property
    def _llm_type(self) -> str:
        """Return the type of chat model."""
        return "zhipuai"

    def _ask_remote(self, messages, streaming=False, **kwargs):
        # 从langchain消息格式，转换到智谱AI输入的格式
        dict_zhipu = [convert_message_to_dict(m) for m in messages]
        
        response = self.client.chat.completions.create(
            model="glm-4",
            messages=dict_zhipu,
            stream=streaming,
            **kwargs
        )

        # 从智谱AI输出的格式，转换到langchain的消息格式
        if not isinstance(response, dict):
            response = response.dict()
        return [convert_dict_to_message(c["message"]) for c in response["choices"]]

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        stream: Optional[bool] = None,
        **kwargs: Any,
    ) -> ChatResult:
        """实现 ZhiputAI 的同步调用"""

        # 问智谱AI，并得到回复
        responses = self._ask_remote(messages, streaming=False, **kwargs)

        return ChatResult(
            generations=[ChatGeneration(message=m) for m in responses]
        )

#### （3）用起来！

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个中英互译机器人，只负责翻译，不要试图对问题做解答。我说中文你就直接翻译成英文，我说英文你就直接翻译为中文。不要输出其他，不要啰嗦。"),
    ("human", "你好"),
    ("ai", "hello"),
    ("human", "{question}"),
])
llm_zhipu = MiniZhipuAI()
chain = prompt | llm_zhipu

chain.invoke({"question": "The competition between China and the United States in the AI field is very intense. Can China catch up?"})

### ✍️ 尝试在智能体中使用

#### （1）定义一个简单工具

In [ ]:
from langchain.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool, convert_to_openai_function
import re

@tool
def ask_neighber(query: str) -> str:
    """我是马冬梅的邻居老大爷，关于她的事情你可以问我"""
    if(re.search("马冬梅", query)):
        return "楼上322"
    else:
        return "我不清楚"

#### （2）作为 openai 风格的回调工具使用

In [ ]:
llm_zhipu = MiniZhipuAI().bind(tools=[convert_to_openai_tool(ask_neighber)])

In [ ]:
llm_zhipu.invoke("告诉我马冬梅在哪个房间？")

In [ ]:
ask_neighber.invoke({"query":"马冬梅"})

#### （3）集成到智能体

In [ ]:
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain import hub

def create_neighber(llm):
    tools = [ask_neighber]
    prompt = hub.pull("hwchase17/openai-tools-agent")
    agent = create_openai_tools_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
create_neighber(MiniZhipuAI()).invoke({"input":"马冬梅住哪里"})

### ✍️ 现在可以轻松切换智能体中的大模型（假装我对 langchian 很满意！👍👍👍）

#### （1）使用 langchain_zhipu

<div class="alert alert-warning">
    <b>⚠️ 思考：</b><br>
    上面代码已经相对完整实现了一个 langchain 大模型；但缺少很多细节控制，可以尝试自己动手添加！    
</div>

**🌞 参考：**
- [🔗 查看 langchain_zhpu 中的实现源码](https://github.com/arcstep/langchain_zhipuai/blob/e55af13eed673bc409ffdb143030e6cc0b2af27c/langchain_zhipu/chat.py#L304-L354) [![PyPI version](https://img.shields.io/pypi/v/langchain_zhipu.svg)](https://pypi.org/project/langchain_zhipu/)

In [ ]:
from langchain_zhipu import ChatZhipuAI

create_neighber(ChatZhipuAI()).invoke({"input":"马冬梅住哪里"})

#### （2）使用 langchain_openai

**这与直接使用OpenAI类似：**

In [ ]:
from langchain_openai import ChatOpenAI

create_neighber(ChatOpenAI()).invoke({"input":"马冬梅住哪里"})

# ❤️ 知识点小结

1. 如果要集成自己的大模型到 langchain ，从 `BaseChatModel` 继承是一个很好的起点
2. BaseChatModel 至少要求你实现 `_generate` 方法，如果补充 `_stream` 方法，就可以提供到全面能力
3. 使用 `langchain_community.adapters.openai` 可以转换 OpenAI 风格的消息格式
4. `ChatPromptTemplate.from_messages` 可以使用语法糖： system, human（或user）, ai（或assistant）等
5. 值得记住几个从源码观察到的实用方法：`get_graph().print_ascii()`，`input_schema.schema()` 等